# Neural network based regression - Densely connected network with clustering

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(""), ".."))

from torch import nn, optim

from nnbma.networks import DenselyConnected, PolynomialNetwork, MergingNetwork
from nnbma.layers import PolynomialExpansion
from nnbma.learning import LearningParameters, MaskedMSELoss, learning_procedure

from helpers.preprocessing import (
    prepare_data,
    prepare_data_transformers,
    prepare_clusters,
    get_names,
)
from helpers.results import save_results

In [ ]:
filename = os.path.join(
    os.path.splitext(os.path.abspath(""))[0], "out-nn-regression-dense"
)

### Dataset

In [ ]:
lines = None  # If None, select all lines by default

In [ ]:
train_set, test_set, train_mask_set, test_mask_set = prepare_data(lines=lines)

n_inputs, n_outputs = train_set.n_inputs, train_set.n_outputs
inputs_names, outputs_names = train_set.inputs_names, train_set.outputs_names

In [ ]:
inputs_transformer, _, __, outputs_transformer = prepare_data_transformers(train_set)

train_set = train_set.apply_transf(inputs_transformer, outputs_transformer)
test_set = test_set.apply_transf(inputs_transformer, outputs_transformer)

### Clusters

`n_comps` is the number of principal components from `clustering.ipynb`.

In [ ]:
n_clusters = 4
n_comps = [500, 100, 75, 350]

### Architecture settings (can be modified)

In [ ]:
n_inputs, n_outputs = len(inputs_names), len(outputs_names)

n_layers = 12
growing_factor = 0.25
poly_degree = 3

In [ ]:
## Architecture hyperparameters

activation = nn.ELU()
batch_norm = False

subnetworks = []
w = n_outputs * [1.0]

for k in range(n_clusters):
    _lines = prepare_clusters(n_clusters)[k + 1]
    n_comp = n_comps[k]

    _, _outputs_names = get_names(lines=_lines)
    _n_outputs = len(_outputs_names)

    n_expanded_inputs = PolynomialExpansion.expanded_features(poly_degree, n_inputs)

    subnetworks.append(
        PolynomialNetwork(
            n_inputs,
            poly_degree,
            DenselyConnected(
                n_expanded_inputs,
                _n_outputs,
                n_layers=n_layers,
                growing_factor=growing_factor,
                activation=activation,
                batch_norm=batch_norm,
                outputs_names=_outputs_names,
            ),
            inputs_names=inputs_names,
            outputs_names=_outputs_names,
        )
    )

    for i in range(n_outputs):
        line = outputs_names[i]
        if line in _outputs_names:
            w[i] = n_outputs / _n_outputs

use_mask = True

## Network creation

model = MergingNetwork(
    subnetworks,
    inputs_names=inputs_names,
    outputs_names=outputs_names,
    inputs_transformer=inputs_transformer,
    outputs_transformer=outputs_transformer,
)

### Training settings (can be modified)

In [ ]:
# Epochs
epochs = 200

# Batch size
batch_size = 500

# Loss function
use_mask = True
loss = MaskedMSELoss() if use_mask else nn.MSELoss()

# Optimizer
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), learning_rate)

# Scheduler
factor = 0.9
patience = 5
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=patience, factor=factor, min_lr=learning_rate * 1e-3
)

In [ ]:
## Learning hyperparameters

learning_params = LearningParameters(loss, epochs, batch_size, optimizer, scheduler)

### Training procedure

In [ ]:
results = learning_procedure(
    model,
    (train_set, test_set),
    learning_params,
    mask_dataset=(train_mask_set, test_mask_set),
)

### Saving results

In [ ]:
plot_profile = True

In [ ]:
arch_name = f"layers_{n_layers}_factor_{growing_factor}_deg_{poly_degree}"

save_results(
    results,
    outputs_names,
    model,
    learning_params,
    use_mask,
    filename,
    architecture_name=arch_name,
    plot_profiles=plot_profile,
)